In [125]:
import os
from bs4 import BeautifulSoup
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
!pip install contractions
import contractions
from collections import defaultdict
from tqdm.notebook import tqdm as tqdm
import ast
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [126]:
from google.colab import drive
drive.mount('/content/drive')
loc = "/content/drive/MyDrive/text_files/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [127]:
def bold_text(text):
  return "\033[4m" + text + "\033[0m"

In [128]:
def preprocess_file(file_path, i):
    with open(file_path, 'r') as file:
        # Read the file
        text = file.read()
        t1 = text

        # Lowercase the text
        text = text.lower()
        t2 = text

        # Remove HTML tags using BeautifulSoup
        soup = BeautifulSoup(text, "html.parser")
        text = soup.get_text()
        t3 = text

        # Perform tokenization
        tokens = word_tokenize(text)
        t4 = tokens

        # Remove stopwords and punctuations
        tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
        t5 = tokens

        # Remove blank space tokens
        tokens = [word for word in tokens if word.strip()]
        t6 = tokens

        if(i>=500 and i<505):
          print()
          print(bold_text("Before lowercasing:")+ f" {t1}")
          print()
          print(bold_text("After lowercasing:")+ f" {t2}")
          print()
          print(bold_text("Before tokenization:")+ f" {t3}")
          print()
          print(bold_text("After tokenization:")+ f" {t4}")
          print()
          print(bold_text("Before removing stopwords and punctuations:") + f" {t4}")
          print()
          print(bold_text("After removing stopwords and punctuations:") + f" {t5}")
          print()
          print(bold_text("Before removing blank space:") + f" {t5}")
          print()
          print(bold_text("After removing blank space:") + f" {t6}")
          print()
          print("************************************************")

        return tokens

In [129]:
def pp_files(file_path, content):
  with open(file_path, 'w') as f:
    f.write(content)

In [130]:
# Process files named file1.txt through file999.txt
loc2 = "/content/drive/MyDrive/IR_Dataset/"
stop_words = set(stopwords.words('english'))
preprossed_files = []
for i in range(1, 1000):
    file_name = loc+f"file{i}.txt"
    tokens = preprocess_file(file_name, i)
    tokens = " ".join(tokens)
      # print(tokens)

    preprossed_fileName = loc2+f"pp_file{i}.txt"
    pp_files(preprossed_fileName, tokens)


<ipython-input-128-9cac72885ed1>:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")



Before lowercasing: Outstanding case!  My H6 fits into this case perfectly.  It is made of heavy duty plastic, which is at least twice as thick at the case that comes with the H6.  Each cut-out within the foam interior is custom sized to fit your H6 and all of the accessories.  The latches are very sturdy.  See picture to get a sense of the size in relation to the standard H6 case.  BUY IT...

After lowercasing: outstanding case!  my h6 fits into this case perfectly.  it is made of heavy duty plastic, which is at least twice as thick at the case that comes with the h6.  each cut-out within the foam interior is custom sized to fit your h6 and all of the accessories.  the latches are very sturdy.  see picture to get a sense of the size in relation to the standard h6 case.  buy it...

Before tokenization: outstanding case!  my h6 fits into this case perfectly.  it is made of heavy duty plastic, which is at least twice as thick at the case that comes with the h6.  each cut-out within the 

In [166]:
#Question2
loc2 = "/content/drive/MyDrive/IR_Dataset/"
def do_pp(text):
  text = text.lower()
  tokens = word_tokenize(text)
  tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
  tokens = [word for word in tokens if word.strip()]
  return tokens

class InvertedIndex:
    def __init__(self):
        self.index = {}

    def index_document(self, document_name, text):
        # Preprocess the text
        words = preprocess_file(loc2+filename, 0)

        # Update the index
        for word in words:
            if word not in self.index:
                self.index[word] = {document_name}
            else:
                self.index[word].add(document_name)

    def save(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self.index, f)

    def load(self, filename):
        with open(filename, 'rb') as f:
            self.index = pickle.load(f)

    def query(self, query):
        # Split the query into words and operators
        elements = re.split(r' (AND|OR|NOT) ', query)

        # Start with the set of all documents
        result = set(self.index.keys())

        # Apply each word/operator to the result
        i = 0
        while i < len(elements):
            if elements[i] == 'AND':
                if i + 1 < len(elements) and elements[i + 1] == 'NOT':
                    temp = set(self.index.keys()) - self.index.get(elements[i + 2], set())
                    result = result & temp
                    i += 3
                else:
                    result = result & self.index.get(elements[i + 1], set())
                    i += 2
            elif elements[i] == 'OR':
                if i + 1 < len(elements) and elements[i + 1] == 'NOT':
                    temp = set(self.index.keys()) - self.index.get(elements[i + 2], set())
                    result = result | temp
                    i += 3
                else:
                    result = result | self.index.get(elements[i + 1], set())
                    i += 2
            elif elements[i] == 'NOT':
                result = result - self.index.get(elements[i + 1], set())
                i += 2
            else:
                if elements[i] in self.index:
                    result = set(self.index[elements[i]])
                else:
                    result = set()
                i += 1

        return result


In [167]:
def execute_queries(index, queries):
    for i, (sequence, operations) in enumerate(queries, 1):
        sequence = re.sub(r'\W+', ' ', sequence.lower())
        words = sequence.split()

        # Construct the query
        query = []
        for word, operation in zip(words, operations.split(', ')):
            query.append(word)
            query.append(operation)
        query.append(words[-1])

        # Execute the query
        result = index.query(' '.join(query))

        # Print the results
        print(f'Query {i}: {" ".join(query)}')
        print(f'Number of documents retrieved for query {i}: {len(result)}')
        print(f'Names of the documents retrieved for query {i}: {", ".join(result)}')

In [168]:
# Directory containing the text files
directory = loc2

index = InvertedIndex()
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        with open(os.path.join(directory, filename), 'r') as f:
            text = f.read()
            index.index_document(filename, text)

# Save the index to a file
index.save('index.pkl')

# Load the index from the file
index.load('index.pkl')



<ipython-input-128-9cac72885ed1>:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [170]:
# Read the number of queries from the user
N = int(input())

# Read the queries from the user
queries = []
for _ in range(N):
    sequence = input()
    operations = input()
    queries.append((sequence, operations))

# Execute the queries
execute_queries(index, queries)

1
guitar


KeyboardInterrupt: Interrupted by user

In [189]:
#Question3
class PositionalIndex:
    def __init__(self):
        self.index = {}

    def index_document(self, document_name, text):
        # Preprocess the text
        words = do_pp(text)

        # Update the index
        for position, word in enumerate(words):
            if word not in self.index:
                self.index[word] = {document_name: [position]}
            else:
                if document_name in self.index[word]:
                    self.index[word][document_name].append(position)
                else:
                    self.index[word][document_name] = [position]

    def save(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self.index, f)

    def load(self, filename):
        with open(filename, 'rb') as f:
            self.index = pickle.load(f)

    def query(self, query):
        # Split the query into words
        words = do_pp(query)
        print(words)

        # Start with the set of all positions of the first word
        result = self.index.get(words[0], {})

        # Apply each word to the result
        for i in range(1, len(words)):
            word_positions = self.index.get(words[i], {})
            for doc in list(result.keys()):
                positions = [pos - i for pos in word_positions.get(doc, [])]
                result[doc] = [pos for pos in result[doc] if pos in positions]

        # Filter out the documents where the query does not exist
        result = {doc: positions for doc, positions in result.items() if positions}

        return result

# Directory containing the text files
directory = loc2

# Create a positional index of the dataset
index = PositionalIndex()
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        with open(os.path.join(directory, filename), 'r') as f:
            text = f.read()
            index.index_document(filename, text)

# Save the index to a file
index.save('index2.pkl')

# Load the index from the file
index.load('index2.pkl')
print(index.index["coffee"])
print(len(index.index["coffee"]))


{'pp_file157.txt': [30], 'pp_file886.txt': [17]}
2


In [191]:
# Read the number of queries from the user
N = int(input())

# Read the queries from the user and execute them
for i in range(N):
    query = input()
    result = index.query(query)
    print(f'Number of documents retrieved for query {i+1} using positional index: {len(result)}')
    print(f'Names of documents retrieved for query {i+1} using positional index: {", ".join(result.keys())}')
#everything acoustic bass ukuleles know smaller model available ukes violins

1
everything acoustic bass ukuleles know smaller model available ukes violins
['everything', 'acoustic', 'bass', 'ukuleles', 'know', 'smaller', 'model', 'available', 'ukes', 'violins']
Number of documents retrieved for query 1 using positional index: 1
Names of documents retrieved for query 1 using positional index: pp_file3.txt
